In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Liver_Cancer"
cohort = "GSE164760"

# Input paths
in_trait_dir = "../../input/GEO/Liver_Cancer"
in_cohort_dir = "../../input/GEO/Liver_Cancer/GSE164760"

# Output paths
out_data_file = "../../output/preprocess/Liver_Cancer/GSE164760.csv"
out_gene_data_file = "../../output/preprocess/Liver_Cancer/gene_data/GSE164760.csv"
out_clinical_data_file = "../../output/preprocess/Liver_Cancer/clinical_data/GSE164760.csv"
json_path = "../../output/preprocess/Liver_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Molecular characterization of hepatocellular carcinoma in patients with non-alcoholic steatohepatitis"
!Series_summary	"Non-alcoholic steatohepatitis (NASH)-related hepatocellular carcinoma (HCC) is increasing globally, but its molecular features are not well defined. We aimed to identify unique molecular traits characterizing NASH-HCC compared to other HCC aetiologies."
!Series_summary	"We collected 80 NASH-HCC and 125 NASH samples from 5 institutions. Expression array (n=53 NASH-HCC; n=74 NASH) and whole exome sequencing (n=50 NASH-HCC) data were compared to HCCs of other aetiologies (n=184). Three NASH-HCC mouse models were analysed with RNAseq/expression-array (n=20). ACVR2A was silenced in HCC cells and proliferation assessed by MTT and colony formation assays."
!Series_summary	"Mutational profiling of NASH-HCC tumours revealed TERT-promoter (56%), CTNNB1 (28%), TP53 (18%) and ACVR2A (10%) as the most-frequently mutated genes. ACVR2A mutation

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# This appears to be a gene expression dataset studying NASH-related HCC
# The Series_summary mentions "Expression array" and "whole exome sequencing"
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# Looking at the sample characteristics dictionary

# 2.1 Trait (Liver Cancer)
# From the characteristics dictionary, we can identify tissue type in row 0
# This can be used to determine liver cancer status
trait_row = 0

def convert_trait(value):
    if value is None:
        return None
    
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Based on the unique values, map to binary (1 for HCC tumor, 0 for non-tumor)
    if "NASH-HCC tumor" in value:
        return 1
    elif any(tissue_type in value for tissue_type in ["Healthy liver", "Cirrhotic liver", "NASH liver", "Non-tumoral NASH liver"]):
        return 0
    return None

# 2.2 Age data
# There is no age information available in the characteristics dictionary
age_row = None

def convert_age(value):
    # Since age data is not available, this function won't be used
    # But we define it for completeness
    if value is None:
        return None
    
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except:
        return None

# 2.3 Gender data
# There is no gender information available in the characteristics dictionary
gender_row = None

def convert_gender(value):
    # Since gender data is not available, this function won't be used
    # But we define it for completeness
    if value is None:
        return None
    
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    
    if value in ['female', 'f']:
        return 0
    elif value in ['male', 'm']:
        return 1
    return None

# 3. Save Metadata
# Trait data is available (trait_row is not None)
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # We have the trait data available, so we can extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Save the clinical data to CSV
    selected_clinical_df.to_csv(out_clinical_data_file, index=True)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features:
{'GSM5018268': [0.0], 'GSM5018269': [0.0], 'GSM5018270': [0.0], 'GSM5018271': [0.0], 'GSM5018272': [0.0], 'GSM5018273': [0.0], 'GSM5018274': [0.0], 'GSM5018275': [0.0], 'GSM5018276': [0.0], 'GSM5018277': [0.0], 'GSM5018278': [0.0], 'GSM5018279': [0.0], 'GSM5018280': [0.0], 'GSM5018281': [0.0], 'GSM5018282': [0.0], 'GSM5018283': [0.0], 'GSM5018284': [0.0], 'GSM5018285': [0.0], 'GSM5018286': [0.0], 'GSM5018287': [0.0], 'GSM5018288': [0.0], 'GSM5018289': [0.0], 'GSM5018290': [0.0], 'GSM5018291': [0.0], 'GSM5018292': [0.0], 'GSM5018293': [0.0], 'GSM5018294': [0.0], 'GSM5018295': [0.0], 'GSM5018296': [0.0], 'GSM5018297': [0.0], 'GSM5018298': [0.0], 'GSM5018299': [0.0], 'GSM5018300': [0.0], 'GSM5018301': [0.0], 'GSM5018302': [0.0], 'GSM5018303': [0.0], 'GSM5018304': [0.0], 'GSM5018305': [0.0], 'GSM5018306': [0.0], 'GSM5018307': [0.0], 'GSM5018308': [0.0], 'GSM5018309': [0.0], 'GSM5018310': [0.0], 'GSM5018311': [0.0], 'GSM5018312': [0.0], 'GSM5018313': [

### Step 3: Gene Data Extraction

In [4]:
# Check if the dataset contains gene expression data based on previous assessment
if not is_gene_available:
    print("This dataset does not contain gene expression data (only miRNA data).")
    print("Skipping gene expression data extraction.")
else:
    # Get the matrix file directly rather than using geo_get_relevant_filepaths
    files = os.listdir(in_cohort_dir)
    if len(files) > 0:
        matrix_file = os.path.join(in_cohort_dir, files[0])
        print(f"Matrix file found: {matrix_file}")
        
        try:
            # Extract gene data
            gene_data = get_genetic_data(matrix_file)
            print(f"Gene data shape: {gene_data.shape}")
            
            # Print the first 20 gene/probe identifiers
            print("First 20 gene/probe identifiers:")
            print(gene_data.index[:20])
        except Exception as e:
            print(f"Error extracting gene data: {e}")
    else:
        print("No files found in the input directory.")


Matrix file found: ../../input/GEO/Liver_Cancer/GSE164760/GSE164760_family.soft.gz


Error extracting gene data: Marker '!series_matrix_table_begin' not found in the file.


### Step 4: Gene Identifier Review

In [5]:
# Observing the gene identifiers in the gene expression data
# These identifiers (e.g., '11715100_at', '11715101_s_at') are not human gene symbols
# They appear to be probe IDs from a microarray platform, likely Affymetrix
# These will need to be mapped to standard human gene symbols for analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Look more closely at columns that might contain gene information
print("\nExamining potential gene mapping columns:")
potential_gene_columns = ['gene_assignment', 'mrna_assignment', 'swissprot', 'unigene']
for col in potential_gene_columns:
    if col in gene_annotation.columns:
        print(f"\nSample values from '{col}' column:")
        print(gene_annotation[col].head(3).tolist())



Gene annotation preview:
Columns in gene annotation: ['ID', 'GeneChip Array', 'Species Scientific Name', 'Annotation Date', 'Sequence Type', 'Sequence Source', 'Transcript ID(Array Design)', 'Target Description', 'Representative Public ID', 'Archival UniGene Cluster', 'UniGene ID', 'Genome Version', 'Alignments', 'Gene Title', 'Gene Symbol', 'Chromosomal Location', 'GB_LIST', 'SPOT_ID', 'Unigene Cluster Type', 'Ensembl', 'Entrez Gene', 'SwissProt', 'EC', 'OMIM', 'RefSeq Protein ID', 'RefSeq Transcript ID', 'FlyBase', 'AGI', 'WormBase', 'MGI Name', 'RGD Name', 'SGD accession number', 'Gene Ontology Biological Process', 'Gene Ontology Cellular Component', 'Gene Ontology Molecular Function', 'Pathway', 'InterPro', 'Trans Membrane', 'QTL', 'Annotation Description', 'Annotation Transcript Cluster', 'Transcript Assignments', 'Annotation Notes']
{'ID': ['11715100_at', '11715101_s_at', '11715102_x_at', '11715103_x_at', '11715104_s_at'], 'GeneChip Array': ['Human Genome HG-U219 Array', 'Human 

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the relevant columns for mapping probe IDs to gene symbols
# Based on the annotation preview, we need to map 'ID' to 'Gene Symbol'
probe_id_col = 'ID'  # Column containing the probe identifiers
gene_symbol_col = 'Gene Symbol'  # Column containing the gene symbols

print(f"Mapping probes from '{probe_id_col}' to genes in '{gene_symbol_col}'")

# 2. Get the mapping dataframe using the identified columns
gene_mapping = get_gene_mapping(gene_annotation, prob_col=probe_id_col, gene_col=gene_symbol_col)
print(f"Gene mapping dataframe shape: {gene_mapping.shape}")
print("Sample of gene mapping:")
print(gene_mapping.head())

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
# Reconfirm the genetic data is extracted correctly
genetic_data = get_genetic_data(matrix_file)
print(f"Genetic data shape before mapping: {genetic_data.shape}")

# Apply the mapping to convert probe-level to gene-level expression
gene_data = apply_gene_mapping(genetic_data, gene_mapping)
print(f"Gene data shape after mapping: {gene_data.shape}")
print("Sample of gene expression data:")
print(gene_data.iloc[:5, :5])

# Save the gene expression data
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Mapping probes from 'ID' to genes in 'Gene Symbol'


Gene mapping dataframe shape: (49384, 2)
Sample of gene mapping:
              ID       Gene
0    11715100_at   HIST1H3G
1  11715101_s_at   HIST1H3G
2  11715102_x_at   HIST1H3G
3  11715103_x_at  TNFAIP8L1
4  11715104_s_at      OTOP2


Genetic data shape before mapping: (49386, 170)


Gene data shape after mapping: (19521, 170)
Sample of gene expression data:
       GSM5018268  GSM5018269  GSM5018270  GSM5018271  GSM5018272
Gene                                                             
A1BG    26.193085   27.247516   22.545718   23.498937   22.978236
A1CF   221.676246  219.701775   89.646004  143.134513  150.538559
A2BP1  206.435882  244.144349  262.732617  274.779167  191.588350
A2LD1   14.832444   18.875513   19.204097   18.426530   14.465543
A2M    809.682278  238.468518  291.242092  635.645959  908.507654


Gene expression data saved to ../../output/preprocess/Liver_Cancer/gene_data/GSE164760.csv


### Step 7: Data Normalization and Linking

In [8]:
# Load the clinical data from the file we saved in step 2
clinical_data_file = out_clinical_data_file
if os.path.exists(clinical_data_file):
    selected_clinical_df = pd.read_csv(clinical_data_file, index_col=0)
    print(f"Loaded clinical data from {clinical_data_file}")
    print(f"Clinical data shape: {selected_clinical_df.shape}")
    print("Clinical data preview:")
    print(selected_clinical_df.head())
else:
    print(f"Clinical data file {clinical_data_file} not found. Re-extracting clinical features...")
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    print("Re-extracted clinical data preview:")
    print(preview_df(selected_clinical_df))

# 1. Normalize gene symbols in the index
print("\nNormalizing gene symbols...")
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
print("First 10 gene identifiers after normalization:")
print(normalized_gene_data.index[:10].tolist())

# Save the normalized gene data to CSV
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Link the clinical and genetic data
print("\nLinking clinical and genetic data...")
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, first 5 columns):")
print(linked_data.iloc[:5, :5])

# 3. Handle missing values in the linked data
print("\nHandling missing values...")
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 4. Determine if the trait and demographic features are biased
print("\nChecking for bias in trait and demographic features...")
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Conduct final quality validation and save relevant information
print("\nConducting final quality validation...")
is_gene_available = len(normalized_gene_data) > 0
is_trait_available = True  # We've confirmed trait data is available in previous steps

note = "This dataset contains gene expression data from NASH-HCC tumors vs. healthy/NASH/cirrhotic livers, appropriate for liver cancer analysis."

is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 6. Save the linked data if it's usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Linked data not saved as dataset is not usable for the current trait study.")

Loaded clinical data from ../../output/preprocess/Liver_Cancer/clinical_data/GSE164760.csv
Clinical data shape: (1, 170)
Clinical data preview:
              GSM5018268  GSM5018269  GSM5018270  GSM5018271  GSM5018272  \
Liver_Cancer         0.0         0.0         0.0         0.0         0.0   

              GSM5018273  GSM5018274  GSM5018275  GSM5018276  GSM5018277  ...  \
Liver_Cancer         0.0         0.0         0.0         0.0         0.0  ...   

              GSM5018428  GSM5018429  GSM5018430  GSM5018431  GSM5018432  \
Liver_Cancer         1.0         1.0         1.0         1.0         1.0   

              GSM5018433  GSM5018434  GSM5018435  GSM5018436  GSM5018437  
Liver_Cancer         1.0         1.0         1.0         1.0         1.0  

[1 rows x 170 columns]

Normalizing gene symbols...
Gene data shape after normalization: (19298, 170)
First 10 gene identifiers after normalization:
['A1BG', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS', 'AACS']


Normalized gene data saved to ../../output/preprocess/Liver_Cancer/gene_data/GSE164760.csv

Linking clinical and genetic data...
Linked data shape: (170, 19299)
Linked data preview (first 5 rows, first 5 columns):
            Liver_Cancer       A1BG        A1CF         A2M      A2ML1
GSM5018268           0.0  26.193085  221.676246  809.682278  47.181279
GSM5018269           0.0  27.247516  219.701775  238.468518  65.398284
GSM5018270           0.0  22.545718   89.646004  291.242092  49.127122
GSM5018271           0.0  23.498937  143.134513  635.645959  54.785169
GSM5018272           0.0  22.978236  150.538559  908.507654  37.466176

Handling missing values...


Linked data shape after handling missing values: (170, 19299)

Checking for bias in trait and demographic features...
For the feature 'Liver_Cancer', the least common label is '1.0' with 53 occurrences. This represents 31.18% of the dataset.
The distribution of the feature 'Liver_Cancer' in this dataset is fine.


Conducting final quality validation...
A new JSON file was created at: ../../output/preprocess/Liver_Cancer/cohort_info.json


Linked data saved to ../../output/preprocess/Liver_Cancer/GSE164760.csv
